# CodeCraft AI – Interactive Demo

This notebook lets you interact with your deployed AI system via its FastAPI endpoint. Use it to:
- Ask questions and view AI responses
- Test edge cases and model behavior
- Showcase end-to-end integration for portfolio or stakeholder demos

> **Note:** Ensure your API is running and accessible (locally or via AWS) before running the cells below.

In [3]:
# Add this cell at the top or after your setup cells

import requests

API_URL = "http://localhost:8000/query"
payload = {"query": "What is CodeCraft AI?", "top_k": 3}

response = requests.post(API_URL, json=payload)
print("Status:", response.status_code)
print("Response:", response.json())

Status: 200
Response: {'results': []}


In [2]:
# It is not required for interactive use, but is essential for CI/CD and reproducibility.

# To run all code cells and verify outputs in CI/CD, use:
#   pytest --nbval notebooks/ai_demo.ipynb
# Or, to parameterize and execute with papermill:
#   papermill notebooks/ai_demo.ipynb output.ipynb

# This cell can also be used to inject test parameters or skip slow/demo-only cells in CI.
import os

IS_CI = bool(os.getenv("CI", False))
if IS_CI:
    print("Running in CI mode: notebook will be executed and validated automatically.")
else:
    print("Running in interactive mode: use the notebook UI for exploration and demos.")

# To skip cells in CI, use the following pattern at the top of any cell:
# if IS_CI:
#     return

# For more info, see: https://nbval.readthedocs.io/ and https://papermill.readthedocs.io/

Running in interactive mode: use the notebook UI for exploration and demos.


In [ ]:
# Configuration: API endpoint URL
API_URL = "http://localhost:8000/api/ask"

In [ ]:
import os
from getpass import getpass

# Authentication: API Key or JWT
API_AUTH_TOKEN = os.getenv("CODECRAFT_API_TOKEN")
if not API_AUTH_TOKEN:
    print("No API token found in environment variable CODECRAFT_API_TOKEN.")
    API_AUTH_TOKEN = getpass("Enter your API key or JWT (input hidden): ")

API_AUTH_HEADER = "Authorization"
API_AUTH_PREFIX = (
    "Bearer"  # Change to "Token" or "" if your API expects a different scheme
)

In [ ]:
import requests
from pprint import pprint


def ask_ai(question: str, api_url: str = API_URL, extra_payload: dict = None):
    payload = {"question": question}
    if extra_payload:
        payload.update(extra_payload)
    headers = {}
    if API_AUTH_TOKEN:
        if API_AUTH_PREFIX:
            headers[API_AUTH_HEADER] = f"{API_AUTH_PREFIX} {API_AUTH_TOKEN}"
        else:
            headers[API_AUTH_HEADER] = API_AUTH_TOKEN
    try:
        response = requests.post(api_url, json=payload, headers=headers, timeout=30)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Error: {e}")
        return None

## Example: Ask a Question

In [ ]:
question = "What is the business value of CodeCraft AI?"
result = ask_ai(question)
pprint(result)

## Try Your Own Question

In [ ]:
# Replace with your own question
my_question = "How does the vector store work?"
my_result = ask_ai(my_question)
pprint(my_result)

---
### Advanced: Interactive Widget (Optional)
You can use `ipywidgets` for a more interactive demo experience.

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, Markdown

    def on_ask_clicked(_):
        user_q = question_box.value.strip()
        if not user_q:
            output_box.clear_output()
            with output_box:
                display(Markdown("**Please enter a question.**"))
            return
        output_box.clear_output()
        with output_box:
            display(Markdown(f"**Question:** {user_q}"))
            response = ask_ai(user_q)
            display(Markdown(f"**Response:** {response}"))

    question_box = widgets.Text(
        value="",
        placeholder="Type your question here",
        description="Question:",
        layout=widgets.Layout(width="80%"),
    )
    ask_button = widgets.Button(description="Ask AI", button_style="primary")
    output_box = widgets.Output()
    ask_button.on_click(on_ask_clicked)
    display(widgets.VBox([question_box, ask_button, output_box]))
except ImportError:
    print(
        "ipywidgets not installed. Run 'pip install ipywidgets' for interactive demo."
    )